In [ ]:
import pandas as pd
import folium
import plotly.express as px
import plotly.graph_objects as go 
from plotly.subplots import make_subplots
import plotly.io as pio

In [ ]:
pio.templates.default = 'plotly_white'

In [ ]:
metro_data = pd.read_csv('Delhi-Metro-Network.csv')
metro_data.head()

In [ ]:
metro_data.describe()

In [ ]:
metro_data.isna().sum()

In [ ]:
metro_data.dtypes

In [ ]:
metro_data['Opening Date'] = pd.to_datetime(metro_data['Opening Date'],format="%d-%m-%Y")

In [ ]:
metro_data['Opening Date'].dtypes

In [ ]:
metro_data['Station Name']

In [ ]:
metro_data['Line'].unique()

In [ ]:
line_colors = {
    'Red line': 'red',
    'Blue line': 'blue',
    'Yellow line': 'beige',
    'Green line': 'green',
    'Voilet line': 'purple',
    'Pink line': 'pink',
    'Magenta line': 'darkred',
    'Orange line': 'orange',
    'Rapid Metro': 'cadetblue',
    'Aqua line': 'black',
    'Green line branch': 'lightgreen',
    'Blue line branch': 'lightblue',
    'Gray line': 'lightgray'
}

In [ ]:
delhi_map_with_line = folium.Map(location=[28.7041, 77.1025],zoom_start=11)

In [ ]:
for index, row in metro_data.iterrows():
    line = row['Line']
    color = line_colors.get(line,"black")
    folium.Marker(
        location = [row['Latitude'],row['Longitude']],
        popup = f"{row['Station Name']}",
        tooltip = f"{row['Station Name']},{line}",
        icon=folium.Icon(color=color)
    ).add_to(delhi_map_with_line)

In [ ]:
delhi_map_with_line

In [ ]:
metro_data['Opening Year'] = metro_data['Opening Date'].dt.year

In [ ]:
station_per_year = metro_data['Opening Year'].value_counts().sort_index()

In [ ]:
station_per_year_df = station_per_year.reset_index()
station_per_year_df.columns = ['Year','Number of Stations']
station_per_year_df

In [ ]:
fig = px.bar(station_per_year_df,x='Year',y='Number of Stations',
             title="Number of Metro Stations Opened Each Year in Delhi",
             labels={'Number of Stations':'Number of Stations Opened'},
             text_auto = False,
             color='Year'
             )

fig.update_layout(xaxis_tickangle = -60,xaxis = dict(tickmode='linear'))
fig.update_layout(width=1400,height=600)


fig.show()

In [ ]:
total = metro_data['Station ID'].count()
total

In [ ]:
station_per_line = metro_data['Line'].value_counts()
station_per_line

In [ ]:
total_distance_per_line = metro_data.groupby('Line')['Distance from Start (km)'].max()
avg_distance_per_line = total_distance_per_line / (station_per_line - 1)

line_analysis = pd.DataFrame(
    {
        'Line':station_per_line.index,
        'Number of Stations' : station_per_line.values,
        'AVG Distance Between Stations' : avg_distance_per_line
    }
)

line_analysis = line_analysis.sort_values(by='Number of Stations',ascending=False)

line_analysis.reset_index(drop=True,inplace=True)
line_analysis

In [ ]:
fig = make_subplots(rows=1,cols=2,subplot_titles=('Number of Stations Per Metro Line',
                                                  'Average Distance Between Stations Per Metro Line (km)'),
                                                  horizontal_spacing=0.2)

fig.add_trace(
    go.Bar(y=line_analysis['Line'],x=line_analysis['Number of Stations'],
           orientation='h',name='Number Of Stations',marker_color='crimson'),
           row=1,col=1
)

fig.add_trace(
    go.Bar(y=line_analysis['Line'],x=line_analysis['AVG Distance Between Stations'],
           orientation='h',name='Average Distance (km)',marker_color='navy'),
           row=1,col=2
)

fig.update_xaxes(title_text="Number of Stations", row=1, col=1)
fig.update_xaxes(title_text="Average Distance Between Stations (km)", row=1, col=2)

fig.update_yaxes(title_text="Metro Line",row=1,col=1)
fig.update_layout(height=600,width=1400,title_text='Metro Line Analysis')

fig.show()

In [ ]:
layout_counts = metro_data['Station Layout'].value_counts()

In [ ]:
fig = px.bar(x=layout_counts.index,y=layout_counts.values,
             labels={'x':'Station Layout', 'y': 'Number of Stations'},
             title='Distribution of Delhi Metro Layouts',
             color=layout_counts.index,
             color_continuous_scale='pastel',
             text_auto=True,
             hover_name=layout_counts.index)

fig.update_layout(coloraxis_showscale=False,template="plotly_white")
fig.update_layout(height=400,width=600)

fig.show()

In [ ]:
s = metro_data['Station Layout'].isin(['At-Grade'])
metro_data[s]['Station Name']